In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [1]:
from torch import cuda, bfloat16
import transformers
import os
from torch import cuda, bfloat16
import transformers
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
# bnb_config = transformers.BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_quant_type='nf4',
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_compute_dtype=bfloat16
# )

model_id = 'meta-llama/Llama-2-7b-chat-hf'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# Need auth token for these
hf_auth = os.environ.get('hf_token')
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    # quantization_config=bnb_config,
    device_map='auto',
    use_auth_token=hf_auth
)
model.eval()
print(f"Model loaded on {device}")

/home/hb/myenv/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model loaded on cuda:0


In [2]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "right"

## Table data

In [3]:
import json
from tablular_eval_util import combine_csv_files, split_dataframe, preprocess_data, run_llm_inference

directory = '/home/hb/dataset_bgp/bgp_tab_dataset_test'
combined_df = combine_csv_files(directory)

# Split the DataFrame into smaller chunks
split_size = 20
data_list = split_dataframe(combined_df, split_size)

# Preprocess the data into the required format
formatted_data = [preprocess_data(chunk) for chunk in data_list]

formatted_data_file = 'llm_table_bgp_data_test_20.json'
with open(formatted_data_file, 'w') as f:
    json.dump(formatted_data, f, indent=4)

with open(formatted_data_file, 'r') as f:
    formatted_data = json.load(f)

output_results_file = 'llm_7B_bgp_data_with_outputs_20.json'
run_llm_inference(formatted_data, model, tokenizer, max_length=3050, output_results_file=output_results_file)


Processed 1/3
Processed 2/3
Processed 3/3
[{'instruction': 'The goal for this task is to determine if the data indicates an anomaly. The context, section, and table columns provide important information for identifying the correct anomaly type.', 'input_seg': '[TLE] The context is about BGP data analysis for detecting anomalies. The section is related to a specific time period of BGP monitoring. [TAB] col: | timestamp | asn | num_routes | num_new_routes | num_withdrawals | num_origin_changes | num_route_changes | max_path_length | avg_path_length | max_edit_distance | avg_edit_distance | num_announcements | num_unique_prefixes_announced | anomaly_status | row 1: | 2017-12-12 04:00:00 | 39523 | 0 | 0 | 0 | 0 | 0 | 0 | 0.0 | 0 | 0.0 | 0 | 0 | no anomalies detected | [SEP] row 2: | 2017-12-12 04:05:00 | 39523 | 0 | 0 | 0 | 0 | 0 | 0 | 0.0 | 0 | 0.0 | 0 | 0 | no anomalies detected | [SEP] row 3: | 2017-12-12 04:10:00 | 39523 | 0 | 0 | 0 | 0 | 0 | 0 | 0.0 | 0 | 0.0 | 0 | 0 | no anomalies de

In [10]:
import pandas as pd

def load_true_labels(csv_file):
    """
    Load the true anomaly labels from the CSV file.
    """
    df = pd.read_csv(csv_file)
    true_anomalies = df.loc[df['anomaly_status'].str.contains('anomaly detected', case=False, na=False), 'timestamp'].tolist()
    return set(true_anomalies)

def load_llm_output(LLM_output_list):
    """
    Convert the provided LLM output list to a set of timestamps.
    """
    llm_anomalies = set(LLM_output_list)
    return llm_anomalies

def evaluate_anomalies(true_anomalies, llm_anomalies):
    """
    Evaluate the performance of LLM anomaly detection.
    """
    # True positives, false positives, and false negatives
    tp = len(true_anomalies & llm_anomalies)
    fp = len(llm_anomalies - true_anomalies)
    fn = len(true_anomalies - llm_anomalies)
    
    # Precision, recall, and F1 score
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return precision, recall, f1

# Load true labels and LLM output
true_labels_file = "/home/hb/dataset_bgp/bgp_tab_dataset_test/bgp_features_asn_39523_russian_hijack_with_labels.csv"

llm_output_list = [
    '2017-12-12 04:00:00',
    '2017-12-12 04:10:00',
    '2017-12-12 04:15:00',
    '2017-12-12 04:20:00',
    '2017-12-12 04:25:00',
    '2017-12-12 04:30:00',
    '2017-12-12 04:35:00',
    '2017-12-12 05:45:00',
    '2017-12-12 05:50:00',
    '2017-12-12 05:55:00',
    '2017-12-12 06:00:00',
    '2017-12-12 06:05:00',
    '2017-12-12 06:10:00',
    '2017-12-12 06:15:00',
    '2017-12-12 06:20:00',
    '2017-12-12 07:45:00',
]

true_anomalies = load_true_labels(true_labels_file)
print(true_anomalies)
llm_anomalies = load_llm_output(llm_output_list)

precision, recall, f1 = evaluate_anomalies(true_anomalies, llm_anomalies)

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")


{'2017-12-12 07:10:00', '2017-12-12 04:45:00', '2017-12-12 07:05:00', '2017-12-12 04:40:00', '2017-12-12 04:50:00'}
Precision: 0.00
Recall: 0.00
F1 Score: 0.00


In [10]:
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = f""
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=1528)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] 
The goal for this task is to identify the type of BGP anomaly based on the given data. The context, section, and table columns provide important information for identifying the correct anomaly type.. 
[TLE] The context is about BGP data analysis for detecting anomalies. The section is related to a specific time period of BGP monitoring. [TAB] col: | timestamp | asn | num_routes | num_new_routes | num_withdrawals | num_origin_changes | num_route_changes | max_path_length | avg_path_length | max_edit_distance | avg_edit_distance | num_announcements | num_unique_prefixes_announced | row 1: | 2024-06-10 12:00:00 | 12345 | 100 | 5 | 3 | 1 | 2 | 4 | 3.5 | 2 | 1.5 | 10 | 8 | [SEP] row 2: | 2024-06-10 12:05:00 | 12346 | 95 | 7 | 4 | 0 | 1 | 5 | 3.2 | 1 | 1.7 | 9 | 7 | [SEP] row 3: | 2024-06-10 12:10:00 | 12347 | 102 | 4 | 5 | 2 | 3 | 4.1 | 3 | 2.5 | 11 | 10 |
Based on the data provided, identify the type of BGP anomaly. The possible anomaly types are: route_flapping, outage, prefix_

## DDoS

In [ ]:
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "Create a Python script with the Scapy library to detect DDoS attacks from a PCAP file. The script should read the file, analyze traffic patterns by monitoring packets per second against a set threshold within defined time windows, and reset counters for each time window. It should also log any potential attacks showing the source IP and packet rate. Include a final check for the last time window at the end of the file."
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=712)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

In [ ]:
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "Generate Python code to calculate and visualize 5G network performance KPIs: Total Network Capacity, Capacity per Area, Capacity per Point, Cost per Capacity, Cost per Area, and Surplus per Area. Load data from '5G_Infrastructure/demand_driven_postcode_data_results.csv'. Use keywords: 'capacity', 'cost', 'area', 'numpoints' to identify relevant columns."
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=712)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] Generate Python code to calculate and visualize 5G network performance KPIs: Total Network Capacity, Capacity per Area, Capacity per Point, Cost per Capacity, Cost per Area, and Surplus per Area. Load data from '5G_Infrastructure/demand_driven_postcode_data_results.csv'. Use keywords: 'capacity', 'cost', 'area', 'numpoints' to identify relevant columns. [/INST]  Sure! Here is some Python code that can be used to calculate and visualize 5G network performance KPIs based on the provided CSV file:
```
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the data from the CSV file
df = pd.read_csv('5G_Infrastructure/demand_driven_postcode_data_results.csv')

# Identify the relevant columns
capacity = df['capacity']
cost = df['cost']
area = df['area']
numpoints = df['numpoints']

# Calculate the KPIs
total_network_capacity = capacity.sum()
capacity_per_area = capacity / area
capacity_per_point = capacity / numpoints
cost_per_capacity = cost / capacity


## Multiple Choice
`5G configuration analysis v1`:
Accuracy: 66%
Correct answers: 13 + 7 = 20
Incorrect answers: 10

`5G configuration analysis v2`:
Accuracy: 56%
Correct answers: 17
Incorrect answers: 13